In [119]:
cd ../analysis/

/Users/akashds1/Dropbox/CrowdSourcing/segmentation/analysis


In [120]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
from collections import defaultdict
import csv

samples_for_num = {
    5: range(10),
    10: range(8),
    15: range(6),
    20: range(4),
    25: range(2),
    30: range(1)
}
all_data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))
# all_data[nworkers][sample_num][objid][clust]
all_algos = ['all', 'MV', 'GT', 'GTLSA', 'basic', 'isoGT', 'isoGTLSA', 'isobasic']
for algo in all_algos:
    if algo == 'all':
        filename = 'pixel_em/all_PRJ_table.csv'
    else:
        filename = 'pixel_em/{}_full_PRJ_table.csv'.format(algo)
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            nworkers = int(float(row['num_workers']))
            sample_num = int(float(row['sample_num']))
            objid = int(float(row['objid']))
            clust = int(float(row['clust']))
    #         print nworkers, sample_num, objid, clust
            all_data[algo][nworkers][sample_num][objid][clust] = row

    obj_missing_noclust = []
    num_present = 0
    for nworkers in samples_for_num.keys():
        for sample_num in samples_for_num[nworkers]:
            for objid in range(1, 48):
                if objid == 35:
                    continue
                if -1 not in all_data[algo][nworkers][sample_num][objid].keys():
                    obj_missing_noclust.append((nworkers, sample_num, objid))
                else:
                    num_present += 1

    print algo, 'missing: ', len(obj_missing_noclust)
    print algo, 'present: ', num_present

all missing:  775
all present:  651
MV missing:  0
MV present:  1426
GT missing:  0
GT present:  1426
GTLSA missing:  0
GTLSA present:  1426
basic missing:  0
basic present:  1426
isoGT missing:  0
isoGT present:  1426
isoGTLSA missing:  0
isoGTLSA present:  1426
isobasic missing:  0
isobasic present:  1426


In [122]:
best_clust = defaultdict(lambda: defaultdict(dict))  # best_clust[nworkers][sample_num][objid] = clust
with open('best_clust_picking.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        nworkers = int(row['num_workers'])
        sample_num = int(row['sample_num'])
        objid = int(row['objid'])
        clust = int(row['clust'])
#         print nworkers, sample_num, objid, clust
        best_clust[nworkers][sample_num][objid] = clust

In [124]:
########## Checking (nworkers, sample, objid) missing -1 clust and with best clust = -1 #########
algo = 'MV'
obj_missing_noclust = []
obj_with_noclust_best = []
num_total = 0
for nworkers in all_data[algo].keys():
    for sample_num in all_data[algo][nworkers]:
        for objid in all_data[algo][nworkers][sample_num]:
            if -1 not in all_data[algo][nworkers][sample_num][objid].keys():
#                 print '({}, {}, {})'.format(nworkers, sample_num, objid)
                obj_missing_noclust.append((nworkers, sample_num, objid))
            if best_clust[nworkers][sample_num][objid] == -1:
                obj_with_noclust_best.append((nworkers, sample_num, objid))
            num_total += 1
print 'num (sample x obj) missing clust = -1 in {}: {}'.format(algo, len(obj_missing_noclust))
print 'num (sample x obj) with best_clust = -1 in {}: {}'.format(algo, len(obj_with_noclust_best))
print 'num total in {}: {}'.format(algo, num_total)

with open('{}_obj_missing_noclust.json'.format(algo), 'w') as fp:
    fp.write(json.dumps(obj_missing_noclust))

num (sample x obj) missing clust = -1 in MV: 0
num (sample x obj) with best_clust = -1 in MV: 827
num total in MV: 1426


In [127]:


def clust_vs_noclust(algo='MV', filtered=False)
    jacc_noclust = defaultdict(list)  # jacc[nworkers] = []
    jacc_bestclust = defaultdict(list)
    for nworkers in all_data[algo].keys():
        for sample_num in all_data[algo][nworkers]:
            for objid in all_data[algo][nworkers][sample_num]:
    #             print nworkers, sample_num, objid
                best = best_clust[nworkers][sample_num][objid]
                if filtered and best == -1 or -1 not in all_data[algo][nworkers][sample_num][objid].keys():
                    continue
                noclust_MV = float(all_data[algo][nworkers][sample_num][objid][-1]['jaccard'])
                jacc_noclust[nworkers].append(noclust_MV)
                bestclust_MV = float(all_data[algo][nworkers][sample_num][objid][best]['jaccard'])
                jacc_bestclust[nworkers].append(bestclust_MV)

    
    x = [5, 10, 15, 20, 25, 30]
    y_noclust = defaultdict(list)
    y_bestclust = defaultdict(list)
    for nworkers in jacc_noclust.keys():
#         x.append(nworkers)
#         print jacc_noclust[nworkers]
#         print jacc_bestclust[nworkers]
        y_noclust.append(np.mean(jacc_noclust[nworkers]))
        y_bestclust.append(np.mean(jacc_bestclust[nworkers]))

    plt.figure()
    plt.title('{}_{}_noclust_vs_bestclust'.format(algo, 'filtered' if filtered else 'unfiltered'))
    plt.plot(x, y_noclust, color='blue', label='noclust')
    plt.plot(x, y_bestclust, color='orange', label='bestclust')
    plt.legend()
    plt.savefig('{}_{}_noclust_vs_bestclust.png'.format(algo, 'filtered' if filtered else 'unfiltered')))
    plt.close()

########## comparing algo jaccs for noclust vs bestclust
for algo in ['MV', 'GT', 'GTLSA', 'basic', 'isoGT', 'isoGTLSA', 'isobasic']:
    for filtered in [True, False]:
        
        

In [128]:
print x
print y_noclust
print y_bestclust

[5, 10, 15, 20, 25, 30]
[0.86751363013787819, 0.86991365399626419, 0.87814695851686642, 0.87871790761739921, 0.87983001131982241, 0.88304977163212539]
[0.88450273489179809, 0.89271183408768962, 0.89442649784486516, 0.89656096954196851, 0.89758237846650757, 0.89895529874264013]
